In [ ]:
from os import getcwd
from os.path import join, abspath, pardir
from json import dump
import geopandas as gp
from shapely.geometry import LineString
import pandas as pd
import numpy as np
from collections import namedtuple
import matplotlib.pyplot as plt

##### Configs

In [ ]:
parent_dir = abspath(join(getcwd(), pardir))
shape_files_dir = join(parent_dir, "2109_STIB_MIVB_Network")
gps_files_dir = join(parent_dir, "GPS")
line_shape_file = join(shape_files_dir, "ACTU_LINES.shp")
stops_shape_file = join(shape_files_dir, "ACTU_STOPS.shp")

##### Load files

In [ ]:
line_df = gp.read_file(line_shape_file)
stops_df = gp.read_file(stops_shape_file)

##### Helper functions for prediction of mode transport

In [ ]:
def get_list_threshold(min=1, max=100):
    return [x for x in range(int(min), int(max)) if all(x % t for t in range(2, x))]

def get_spatial_join(df1, df2, threshold_list, column=None, **kwargs):
    if not isinstance(df1, gp.GeoDataFrame): raise ValueError("Kindly, pass a valid geo pandas dataframe.")
    if not isinstance(df2, gp.GeoDataFrame): raise ValueError("Kindly, pass a valid geo pandas dataframe.")
    if (not isinstance(threshold_list, list)) or (not len(threshold_list)): raise ValueError("Kindly, pass a valid threshold list.")
    
    # Make sure both geo-panda dataframes 'df1' & 'df2' have the same coordinate reference system (crs)
    if df1.crs != df2.crs:
        df1 = df1.to_crs(df2.crs)
    
    _threshold_ = float(0)
    df = None
    for threshold in threshold_list:
        df = df2.sjoin_nearest(df1, max_distance=float(threshold), distance_col="distances", **kwargs)
        if not df.empty:
            _threshold_ = threshold
            break
    if (column is None) or (column not in df.columns.to_list()):
        return _threshold_, list()
    else:
        return _threshold_, list(df[column].unique())
    
def get_intersection(list_1, list_2):
    if (not isinstance(list_1, list)) or (not isinstance(list_2, list)): raise ValueError("Kindly, pass a valid list for intersection.")
    
    return list( set(list_1).intersection( set(list_2) ) )

def plot_tracks(track_df, df, modes, col='LIGNE', title=str()):
    fig, ax = plt.subplots(figsize=(15, 15))
    track_df.to_crs(df.crs).plot(ax=ax, alpha=0.7, color="pink")
    line_df.loc[df[col].isin(modes)].plot(ax=ax)
    fig.suptitle(title)
    

##### Predict the mode of transport for each GPS track

In [ ]:
def predict_transport_mode(gps_df, line_df, stops_df, detailed_analysis=False):
    if not isinstance(gps_df, pd.DataFrame): raise ValueError("Kindly, pass the gps track(s) data as a pandas dataframe.")
    if not isinstance(stops_df, gp.GeoDataFrame): raise ValueError("Kindly, pass the stops data as a geo pandas dataframe.")
    if not isinstance(line_df, gp.GeoDataFrame): raise ValueError("Kindly, pass the stops data as a geo pandas dataframe.")
    
    # Get list of all the tracks in the GPS data
    tracks = list(gps_df['TrackId'].unique())
    
    # Max. threshold for max_dist (in meters)
    max_dist_threshold = float(100)
    
    # List of thresholds for lines and stops
    lines_max_dist_threshold = get_list_threshold(1, max_dist_threshold)
    stops_max_dist_threshold = get_list_threshold(5, max_dist_threshold)
    
    # For result
    result = namedtuple('Result', ['track_id', 'mode', 'modes'])
    
    # For each GPS track
    for track in tracks:
        
        # 1. Get 'track' data from gps_df
        track_df = gps_df.loc[gps_df['TrackId'].isin([track])]
        
        # 2. Convert into geo-pandas dataframe with coordinate reference system (crs) of '4326' (WGS 84)
        track_df = gp.GeoDataFrame(track_df, geometry=gp.points_from_xy(track_df['lon'], track_df['lat']), crs=4326)
        
        # 3. Get spatial join with lines
        line_max_dist_threshold, closest_lines = get_spatial_join(track_df, line_df, threshold_list=lines_max_dist_threshold, column="LIGNE")

        # 4. Get spatial join with stops
        stop_max_dist_threshold, closest_stops = get_spatial_join(track_df, stops_df, threshold_list=stops_max_dist_threshold, column="Code_Ligne")
        
        # 5. Get intersection of common modes
        modes = get_intersection(closest_lines, closest_stops)
        
        # Sanity check (if no modes) -> probably the mode is "others"
        if not len(modes):
            result[track] = Result(track_id=track, mode="others", modes=list())
            continue
        
        # 6. For each transport mode, check how close is it to your track
        # for mode in modes:
        print("Track: {}".format(track))
        plot_tracks(track_df, line_df, modes, 'LIGNE', "Track {}".format(track))
        # plot_tracks(track_df, stops_df, modes, 'Code_Ligne')

            
        # print("Track:\t{}\tmax_line:\t{}\tmax_stop:\t{}\tModes:\t{}".format(track, line_max_dist_threshold, stop_max_dist_threshold, modes))        

In [ ]:
predict_transport_mode(gps_df, line_df, stops_df)

##### Get the GPS Tracks

In [48]:
gps_df = pd.read_csv(join(gps_files_dir, "GPStracks.csv"))
max_dist_threshold = float(100)
lines_max_dist_threshold = get_list_threshold(1, max_dist_threshold)
stops_max_dist_threshold = get_list_threshold(5, max_dist_threshold)

In [49]:
gps_df.head()

,TrackId,lat,lon,time
0,1,50.851152,4.345326,2021-11-16T08:35:24Z
1,1,50.851235,4.345191,2021-11-16T08:35:45Z
2,1,50.851328,4.345069,2021-11-16T08:35:48Z
3,1,50.851403,4.344988,2021-11-16T08:35:51Z
4,1,50.851500,4.344920,2021-11-16T08:35:55Z


In [50]:
tracks = list(gps_df['TrackId'].unique())
tracks

[1, 3, 4, 5, 6, 7, 8, 10, 11]

In [51]:
track = 8

In [52]:
track_df = gps_df.loc[gps_df['TrackId'].isin([track])]

In [53]:
track_df = gp.GeoDataFrame(track_df, geometry=gp.points_from_xy(track_df['lon'], track_df['lat']), crs=4326)

In [54]:
line_max_dist_threshold, closest_lines = get_spatial_join(track_df, line_df, threshold_list=lines_max_dist_threshold, column="LIGNE")

In [55]:
stop_max_dist_threshold, closest_stops = get_spatial_join(track_df, stops_df, threshold_list=stops_max_dist_threshold, column="Code_Ligne")

In [56]:
modes = get_intersection(closest_lines, closest_stops)

In [90]:
lines = line_df.loc[line_df['LIGNE'].isin(modes)]
stops = stops_df.loc[stops_df['Code_Ligne'].isin(modes)]

In [95]:
track_df = track_df.to_crs(line_df.crs)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
track_df.plot(ax=ax, alpha=0.7, color="pink")
line_df.loc[line_df['LIGNE'].isin(modes)].plot(ax=ax)
# line_df.plot(ax=ax)